In [1]:
import torch
import os

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize

In [17]:
import time

In [2]:
def get_encode_length(tokenizer, sentence) :
    encoded = tokenizer(sentence, padding=True, truncation=False)

In [3]:
def get_encode_data(tokenizer, sentence):
    encoded_inputs = tokenizer(sentence, padding=True, truncation=False)
    input_ids = torch.tensor(encoded_inputs['input_ids'])
    attention_masks = torch.tensor(encoded_inputs['attention_mask'])

In [4]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [5]:
def get_model(model_name, device, tokenizer=None) :

    print(model_name)

    if tokenizer is None :
      tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    return model.to(device), tokenizer

In [46]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
        top_p=0.95, # 누적 확률이 top_p %인 후보집합에서만 생성
        top_k=50, # 확률 순위 top_k 밖의 sample은 제외
        do_sample=False #True
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [47]:
def seperate_sentence(text, chunk) :
    ss = sent_tokenize(text)

    sentences = []
    merged = ""

    for s in ss :
        if len(merged) + len(s) <= chunk :
            merged = merged + s
        else :
            sentences.append(merged)
            merged = s

    if len(merged) > 256 :
        sentences.append(merged)
    elif len(merged) > 0 :
        sentences[-1] = sentences[-1] + merged

    return sentences

In [85]:
def summarize_long_sentence(text, chunk, tokenizer, model, device, max_tokens, min_tokens, num_beams, verify=False) :

    ss = seperate_sentence(text, chunk)
    summaries = []

    for s in ss :
        if verify == True :
            print(s)

        summary = summarize(s, tokenizer, model, device, max_tokens, min_tokens, num_beams)
        
        if verify == True:
            print(summary)

        summaries.append(summary)

    return summaries

In [99]:
def run_summarize(text, tokenizer, model, device, chunk=1024, max_tokens=200, min_tokens=100, num_beams=2, verify=False):
    results = summarize_long_sentence(text, chunk, tokenizer, model, device, max_tokens, min_tokens, num_beams, verify=verify)
    summary = "".join(results)

    while len(summary) > chunk :
        results = summarize_long_sentence(summary, chunk, tokenizer, model, device, max_tokens, min_tokens, num_beams, verify=verify)
        summary = "".join(results)

    return summary

In [97]:
def test_summarize(text, tokenizer, model, device, chunk=1024, max_tokens=200, min_tokens=100, num_beams=2, verify=False):
    print('============ original ===============')
    print(text)

    start = time.time()

    summary = summarize(text, tokenizer, model, device, max_tokens, min_tokens, num_beams)

    end = time.time()
    print(f"time to summarize is {end-start:.5f} sec")

    print('\n============  summary  ================')
    print(summary)

    print('========================================')

    start = time.time()

    results = summarize_long_sentence(text, chunk, tokenizer, model, device, max_tokens, min_tokens, num_beams, verify=verify)
    summary = "".join(results)

    print('\n===============  long summary  ================')
    print(summary)
    print('==============================================')

    while len(summary) > chunk :
        results = summarize_long_sentence(summary, chunk, tokenizer, model, device, max_tokens, min_tokens, num_beams, verify=verify)
        summary = "".join(results)

        print('\n===============  long summary2  ================')
        print(summary)
        print('===============================================')

    end = time.time()
    print(f"time to summarize long sentence is {end-start:.5f} sec")

In [18]:
model_name = 'psyche/KoT5-summarization'
max_tokens = 200
min_tokens = 100
num_beams = 2

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('available device: ', device)
else:
    device = torch.device("cpu")
    print('available device: ', device)

available device:  cuda


In [11]:
model, tokenizer = get_model(model_name, device)

psyche/KoT5-summarization


In [74]:
filename='/home/osung/data/korean/modu/json/news_PEST_ksic_score.tsv'

In [75]:
df = pd.read_csv(filename, sep='\t')

In [108]:
len(df)

1530052

In [77]:
df.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')

In [78]:
df.iloc[0].text

'코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코로나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다. 의료 현장에서는 “코로나 확산으로 암 환자 등 일반 중환자 치료에 공백이 생기고 있다”는 목소리가 나온다. 18일 김범석 서울대 혈액종양내과 교수팀이 2019~2020년 서울대병원에서 사망한 암 환자 1456명을 분석한 결과 2019년 서울대병원 응급실에서 세상을 떠난 암 환자는 53명에서 2020년 99명으로 약 86% 대폭 증가한 것으로 나타났다. 말기 암 환자의 응급실 체류 시간도 늘었고, 환자의 생존을 위해 강제로 혈압을 높이는 승압제를 사용한 환자 수도 52.3%에서 59.2%로 약 7%포인트 늘었다. 심폐소생술을 받은 암 환자도 12.5%에서 16.3%로 증가했다. 김 교수는 “코로나 환자도 중요하지만, 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다”고 말했다.'

In [26]:
run_summarize(df.iloc[0].text, tokenizer, model, device)

============ original ===============
코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코로나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나 확산으로 암 환자 등 일반 중환자 치료에 공백이 생기고 있다는 목소리가 나온다 18일 김범석 서울대 혈액종양내과 교수팀이 20192020년 서울대병원에서 사망한 암 환자 1456명을 분석한 결과 2019년 서울대병원 응급실에서 세상을 떠난 암 환자는 53명에서 2020년 99명으로 약 86 대폭 증가한 것으로 나타났다 말기 암 환자의 응급실 체류 시간도 늘었고 환자의 생존을 위해 강제로 혈압을 높이는 승압제를 사용한 환자 수도 523에서 592로 약 7포인트 늘었다 심폐소생술을 받은 암 환자도 125에서 163로 증가했다 김 교수는 코로나 환자도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다


/home/osung/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


464 283
time to summarize is 2.20798 sec
============ summary  ================
코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코코나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나가 확산으로 암 환자 등 일반 중 환자 치료에 공백이 생기고 있다는 목소리가 나온다. 김 교수는 코로나도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다. 코로나의 확산으로 일반 중환자의 치료가 공백이 생긴다는 목소리가 나오고 있다. 김교수는 코로나는 물론 이 때문에 방치된 환자들이 많다는 사실을 알아야 한다 말했다.
464 283
=============== long summary  ================
코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코코나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나가 확산으로 암 환자 등 일반 중 환자 치료에 공백이 생기고 있다는 목소리가 나온다. 김 교수는 코로나도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다. 코로나의 확산으로 일반 중환자의 치료가 공백이 생긴다는 목소리가 나오고 있다. 김교수는 코로나는 물론 이 때문에 방치된 환자들이 많다는 사실을 알아야 한다 말했다.
time to summarize long sentence is 1.32148 sec


In [28]:
run_summarize(df.iloc[1].text, tokenizer, model, device)

============ original ===============
 휴가때 골프 꼭 치고 싶으면 자비로 쳐라 허태열 대통령비서실장이 최근 수석비서관회의를 주재하면서 휴가 때 골프를 꼭 치고 싶은 사람은 문제가 되지 않을 사람과 자비로 쳐도 된다며 웬만하면 필드골프장 대신 스크린골프를 이용하는 게 좋겠다고 말한 것으로 전해졌다 2월 박근혜 정부 출범 이후 골프 금지령이 내려진 적은 없지만 북한의 위협이 이어진 데다 정권 출범의 긴장감이 더해져 공직자들은 자연스레 골프를 칠 엄두를 내지 못했다 이와 관련해 박 대통령은 10일 언론사 논설실장 오찬에서 캐디들도 수입이 그렇고 자꾸 외국만 나가서 어떻게 하느냐고 걱정하는 이야기도 있다 지금 여러 가지 생각을 하고 있다고 밝혔다 청와대 관계자는 자비를 들여 취미로 치는 것까지 문제 삼을 수야 있겠느냐며 그렇다고 이제부터 골프를 쳐도 된다고 선언하는 것도 우습고 자연스럽게 분위기가 잡힐 것이라고 말했다 허 실장의 발언이 알려지자 관가에선 골프를 쳐도 된다는 건지 치지 말라는 건지 헷갈린다는 반응이 나왔다
497 317
time to summarize is 1.45685 sec
============ summary  ================
허태열 대통령비서실장이 최근 수석비서관회의를 주재하면서 휴가 때 골프를 꼭 치고 싶은 사람은 문제가 되지 않을 사람과 자비로 쳐도 된다며 웬만하면 필드골프장 대신 스크린골프를 이용하는 게 좋겠다고 말한 것으로 전해졌다 2월 박근혜 정부 출범 이후 골프 금지령이 내려진 적은 없지만 북한의 위협이 이어진 데다 정권 출범의 긴장감이 더해져 공직자들은 자연스레 골프를 칠 엄두를 내지 못했다 이와 관련해 박 대통령은 10일 언론사 논설실장 오찬에서 캐디들도 수입이 그렇고 자꾸 외국만 나가서 어떻게 하느냐고 걱정하는 이야기도 있다 지금 여러 가지 생각을 하고 있다고 밝혔다
497 317
=============== long summary  ================
허태열 대통령비서실장

In [30]:
df.columns

Index(['Unnamed: 0', 'id', 'topic', 'text', 'ksic', 'max_scores', 'keywords',
       'keywords_orig', 'ksic_scores', 'max_ksic', 'ksic_count',
       'max_ksic_scores'],
      dtype='object')

In [31]:
df.iloc[0]

Unnamed: 0                                                         0
id                                              NWRW2200000004.11879
topic                                                              P
text               코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급...
ksic                                                            c271
max_scores                                                    0.4621
keywords           {'김': 1, '분석': 1, '바이러스': 1, '코로나': 5, '병원': 2...
keywords_orig      {'환자': 11, '코로나': 5, '암': 5, '일반': 3, '중환자': 3...
ksic_scores        {'c102': 1, 'c272': 1, 'c211': 2, 'c271': 24, ...
max_ksic                                                    ['c271']
ksic_count                                                         1
max_ksic_scores                                                   24
Name: 0, dtype: object

In [66]:
df = df.drop(columns='Unnamed: 0')

In [33]:
df2.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores', 'keywords',
       'keywords_orig', 'ksic_scores', 'max_ksic', 'ksic_count',
       'max_ksic_scores'],
      dtype='object')

In [34]:
df = df2

In [79]:
df_sorted = df.sort_values(by='text', key=lambda x: x.str.len(), ascending=False)

In [81]:
df_sorted.iloc[0].text

"[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 되고, 그래서 이런 국면에서 필요한 것이 지난번에 합의했던 여야정 

In [86]:
len(df_sorted.iloc[0].text)

24234

In [90]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 5.79661 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길

902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360

===============  long summary2  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다.제가 분명하게 검찰에 말하고 싶은 것은 공수처도 수사권 조정법안도 그렇고, 지금까지 검찰이 사정기구로 본연의 역할을 다하지 못했기 때문에 개혁의 방안으로 논의가 되는 것이다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다. 민정수석이 가장 중요한 정부에서의 책임들이 인사검증과 권력기관들의 개혁이 제일 중요한 임무 가운데 하나다. 청와대 검증부터 청문회까지 전체가 하나의 검증 과정인 것이다.인사청문회가 정쟁쟁의 장으로 운영된다면 좋은 인사를 발탁하는 장이 아니라 좋은 인사를 막는 장이 될 것이다. 총리, 장관들이 정치에 나선다면 그것은 전적으로 본인 의사에 달려 있는 것이고, 대통령으로서 바라고 싶은 것은 선거에 나갈 생각이 있다면 선거시기에 임박해서가 아니라 충분한 여유를 두고 의사를 밝히는 것이 바람직하겠다고 생각하고 있다. 또한 요즘 유권자들의 요구도, 이제는 낙하산으로 공천 받아 내려오는 게 아니라 지역에서 지역주민과 밀착되길 바라기 때문에 그것이 유권자들 요구라고도 생각한다. 그뿐만 아

In [92]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 6.46697 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.검찰 스스로 개혁을 할 수 있는 많은 기회들을 지

In [94]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 6.44101 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길

902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360

===============  long summary2  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다.제가 분명하게 검찰에 말하고 싶은 것은 공수처도 수사권 조정법안도 그렇고, 지금까지 검찰이 사정기구로 본연의 역할을 다하지 못했기 때문에 개혁의 방안으로 논의가 되는 것이다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다. 민정수석이 가장 중요한 정부에서의 책임들이 인사검증과 권력기관들의 개혁이 제일 중요한 임무 가운데 하나다. 청와대 검증부터 청문회까지 전체가 하나의 검증 과정인 것이다.인사청문회가 정쟁쟁의 장으로 운영된다면 좋은 인사를 발탁하는 장이 아니라 좋은 인사를 막는 장이 될 것이다. 총리, 장관들이 정치에 나선다면 그것은 전적으로 본인 의사에 달려 있는 것이고, 대통령으로서 바라고 싶은 것은 선거에 나갈 생각이 있다면 선거시기에 임박해서가 아니라 충분한 여유를 두고 의사를 밝히는 것이 바람직하겠다고 생각하고 있다. 또한 요즘 유권자들의 요구도, 이제는 낙하산으로 공천 받아 내려오는 게 아니라 지역에서 지역주민과 밀착되길 바라기 때문에 그것이 유권자들 요구라고도 생각한다. 그뿐만 아

In [96]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

24234 446
1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411
1928 296
1928 345


In [59]:
ss = seperate_sentence(text, 1024)

In [60]:
ss

['',
 '전문 문재인 정부 2년 특집 대담 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS 문재인 정부 2년 특집 대담 대통령에게 묻는다에 출연해 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다 다음은 일문일답 취임 2주년 소회 2년 전 오늘 이 시간 기억하실지 모르겠다 나라다운 나라를 만들어 달라 이런 요구 속에서 문재인 정권이 출범을 했다 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다 먼저 국민들께 감사인사부터 드려야겠다 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고 저를 대통령으로 선택해 주셨다 그래서 문재인 정부는 촛불정신 위에 서 있다 촛불민심이 명하는 대로 국정농단 그리고 반칙과 특권이라는 적폐시대를 마감하고 새로운 시대 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다 얼마나 기대에 부응했는지 잘 모르겠다 많은 성과들이 있었다고 생각한다 그러나 한편으로 또 아쉬운 부분들도 많이 있고 또 보완해야 할 과제들도 많이 있다고 느낀다 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다 국내 정치 국내 정치 현안 문제로 들어가야 할 것 같다 조금 전에 대북 현안들 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나 이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다 제가 여쭤본 것은 국내 정치권의 그렇다 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다 처리해야 할 민생법안도 많이 있고 추경 문제도 논의해야 되고 그래서 이런 국면에서 필요한 것이 지난번에 합의했던 여야정 상설국정협의체를 가동하는 것이라고 생각한다 우선적으로 방금 현안으로 대두된

In [61]:
len(ss)

2

In [100]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411
1928 296
1928 345


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.한일 관계문제다. 과거사 문제는 과거를 지울 수 없는 문제고 과거사 문제가 한일관계의 족쇄가 된 게 너무 오랜 시간이다. 다만, 어려움을 겪고 있는 것은 과거사 문제로 한번씩 양국관계발전에 발목을 잡고 있는데 그것은 결코 한국정부가 만들어 내고 있는 문제가 아니다. 하지만 일본 정치지도자들이 자꾸 그 문제를 국내 정치적인 문제로 다루기 때문에 미래지향적인 발전의 발목을 잡는 일이 거듭되고 있다고 본다. -제일 문제는 청년일자리다. 제조업의 혁신해 고도화해서 경쟁력을 높여 일자리를 늘리는 방향이 하나 있고, 신산업들을 빨리 성장시켜서 좋은 일자리를 만들어야겠고, 요즘 벤처창업들을 크게 늘리고 지원할 필요가 있다."

In [101]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262
902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360
997 396
991 338
1018 317
1043 311


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다."

In [102]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1536, verify=False)

1517 285
1448 304
1510 269
1505 232
1466 377
1489 361
1399 331
1477 383
1419 354
1414 402
1509 315
1497 362
1493 339
1488 334
1526 283
1633 411
1395 416
1384 392
1358 328
1153 333


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 그 해법을 선택한 것을 가지고 독재라 하는 것은 정말 조금 맞지 않는 얘기란 말씀을 드린다. 검찰 스스로 개혁을 할 수 있는 많은 기회들을 지금까지 놓쳐 왔다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.북한이 며칠 전에 여러 종류의 단거리 발사체를 발사한 데 이어서 오늘은 일단 단거리 미사일로 추정되는 그런 발사를 했다. 유엔 안보리 결의 속에는 탄도미사일을 하지 말라는 그런 표현이 있기 때문에 비록 단거리라 할지라도 그것이 탄도미사일일 경우에는 유엔안보리 결의에 위반될 소지도 없지 않다고 생각한다. 그런 방식으로 북한의 의도를 여러 가지로 해석하게 만들고 또 우려하게 만들고 자칫 잘못하면 대화와 협상국면의 찻물을 끼얹는 이런 성격을 거듭 하는 것은 결코 바람직하지 않다는 것을 북한측에 다시 한 번 이야기하고 싶다. 그런 형편이기 때문에 북한동포들의 심각한 기아상태를 우리가 외면할 수 없고, 우리가 동포애나 인도주의적인 차원에서라도 우리가 북한의 식량을 지원할 필요가 있다고 생각하는 것이다.북한의 식량지원 문제에 대해서 저로서는 가장 바람직한 것은 지금 패스트트랙 문제 때문에 여야 간 정국이 교착상태에 빠져 있는데 그 문제는 별도로 해결하더라도 북한에 대한 식량지원에 대해서는 대통령과 여야가 함께 모여서 협의를 하는 것이 바람직하지 않을까 라는 생각을 하고 있다. 그래서 그렇기 때문에 사실은 식량지원의 문제에 대해서 우리가 한

In [103]:
t = "문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다."

In [105]:
len(t)

311

In [106]:
len("문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.한일 관계문제다. 과거사 문제는 과거를 지울 수 없는 문제고 과거사 문제가 한일관계의 족쇄가 된 게 너무 오랜 시간이다. 다만, 어려움을 겪고 있는 것은 과거사 문제로 한번씩 양국관계발전에 발목을 잡고 있는데 그것은 결코 한국정부가 만들어 내고 있는 문제가 아니다. 하지만 일본 정치지도자들이 자꾸 그 문제를 국내 정치적인 문제로 다루기 때문에 미래지향적인 발전의 발목을 잡는 일이 거듭되고 있다고 본다. -제일 문제는 청년일자리다. 제조업의 혁신해 고도화해서 경쟁력을 높여 일자리를 늘리는 방향이 하나 있고, 신산업들을 빨리 성장시켜서 좋은 일자리를 만들어야겠고, 요즘 벤처창업들을 크게 늘리고 지원할 필요가 있다.")

641

In [107]:
len("문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 그 해법을 선택한 것을 가지고 독재라 하는 것은 정말 조금 맞지 않는 얘기란 말씀을 드린다. 검찰 스스로 개혁을 할 수 있는 많은 기회들을 지금까지 놓쳐 왔다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.북한이 며칠 전에 여러 종류의 단거리 발사체를 발사한 데 이어서 오늘은 일단 단거리 미사일로 추정되는 그런 발사를 했다. 유엔 안보리 결의 속에는 탄도미사일을 하지 말라는 그런 표현이 있기 때문에 비록 단거리라 할지라도 그것이 탄도미사일일 경우에는 유엔안보리 결의에 위반될 소지도 없지 않다고 생각한다. 그런 방식으로 북한의 의도를 여러 가지로 해석하게 만들고 또 우려하게 만들고 자칫 잘못하면 대화와 협상국면의 찻물을 끼얹는 이런 성격을 거듭 하는 것은 결코 바람직하지 않다는 것을 북한측에 다시 한 번 이야기하고 싶다. 그런 형편이기 때문에 북한동포들의 심각한 기아상태를 우리가 외면할 수 없고, 우리가 동포애나 인도주의적인 차원에서라도 우리가 북한의 식량을 지원할 필요가 있다고 생각하는 것이다.북한의 식량지원 문제에 대해서 저로서는 가장 바람직한 것은 지금 패스트트랙 문제 때문에 여야 간 정국이 교착상태에 빠져 있는데 그 문제는 별도로 해결하더라도 북한에 대한 식량지원에 대해서는 대통령과 여야가 함께 모여서 협의를 하는 것이 바람직하지 않을까 라는 생각을 하고 있다. 그래서 그렇기 때문에 사실은 식량지원의 문제에 대해서 우리가 한미간에 합의를 한 것이 이번 발사 이전인데, 그 이후에 또 다시 발사가 있었기 때문에 이 점에 대해서는 국민들의 공감이나 제재가 필요하다고 생각한다. 한일관계는 굉장히 중요하다고 생각하고, 앞으로 더 미래지향적으로 발전돼 나가야 한다고 생각한다.고령화가 급격하게 진행되고 있어서 65세 이상 인구가 14%가 넘는 고령사회를 이미 2017년에 통과했고, 2025년이면 20%가 넘는 초고령사회가 될 것으로 예상된다. 고령인구가 크게 늘고 있기 때문에 일자리 수를 더 늘리고 과거의 급여가 낮았기 때문에 급여를 두배 높여서 실제로 어르신들의 빈곤 해결에 도움되도록 정부가 노력하고 있는 것이다. 거시적인 경제성공은 인정하고 자부심을 가져야 한다고 생각한다. 다만 국민들에게 고르게 다 소득배분이 되고 있지 않고 있기 때문에 아직도 양극화가 심각하고 특히 소득이 낮은 층의 소득이 늘지 않고 있기때문에 그분들의 사는 문제가 해결이 안 되고 있다.")

1469

In [109]:
filename2='/home/osung/data/korean/modu/json/combined_news.tsv'

In [115]:
df2 = pd.read_csv(filename2, sep='\t')

In [116]:
len(df2)

3611424

In [112]:
df2.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')

In [113]:
len(df)

1530052

In [114]:
df.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')